In [1]:
import pandas as pd
import numpy as np
import csv

import warnings
warnings.filterwarnings("ignore")

In [2]:
#data_los = pd.read_excel(r'E:\_Projects\orders\rcv\los_20241007_risk_upside.xlsb', \
#                       sheet_name='pub_line_of_sight_v los_daily_d',engine='pyxlsb')
# data = data_los.copy()               

In [3]:

data_los = pd.read_csv(r'E:\_Projects\los\LOS_Plugin riskupside.csv',low_memory=False)
# data = data_los.copy()               

In [4]:
#Converting to the required format
#*M* mcids

data_los['mfg_config_id_hypen'] = np.where(data_los['mfg_config_id'].str.contains('M'), data_los['mfg_config_id'], '')

#4-4-4 kits, cables
data_los ['mfg_config_id_hypen'] = np.where( ((data_los ['mfg_config_id'].str.len() == 12) & \
                                          (data_los ['mfg_config_id_hypen']=='')), \
    data_los['mfg_config_id'].str[:4] + '-' + data_los['mfg_config_id'].str[4:8] + '-' + data_los['mfg_config_id'].str[8:12], \
                                         data_los ['mfg_config_id_hypen'] \
                                        )
    
#*NCR modules
data_los ['mfg_config_id_hypen'] = np.where( ((data_los ['mfg_config_id'].str.len() == 13) & \
                                          (data_los ['mfg_config_id'].str[11:13]=='NCR')), \
    data_los['mfg_config_id'].str[:3] + '-' + data_los['mfg_config_id'].str[3:10] + '-' + data_los['mfg_config_id'].str[10:14], \
                                         data_los ['mfg_config_id_hypen'] \
                                       )
# data_los = data_los[data_los['mfg_config_id_hypen'].notnull()]
#print(data_los.columns)

Index(['los_date', 'los_line_id', 'instance_id', 'key_account_name',
       'account_nbr', 'account_name', 'region_name', 'area_name',
       'los_category_name', 'los_product_range_name',
       ...
       'project_site_id', 'project_id', 'account_industry_code',
       'account_industry_name', 'offline_revenue_flag',
       'transaction_business_tag_name', 'transaction_company_owner_name',
       'order_line_custom_status_name', 'order_line_flow_status_name',
       'mfg_config_id_hypen'],
      dtype='object', length=277)


In [5]:
data_los_non_balnk = data_los.copy()
data_los_non_balnk = data_los_non_balnk[data_los_non_balnk['pred_rev_risk_yr_month'].notnull()]
#print(data_los_non_balnk.shape)
data_los_balnk = data_los.copy()
data_los_balnk = data_los_balnk[data_los_balnk['pred_rev_risk_yr_month'].isnull() &data_los_balnk['pred_rev_upside_yr_month'].notnull()]
#print(data_los_balnk.shape)

data_los = pd.concat([data_los_non_balnk, data_los_balnk], axis=0)
#print(data_los.shape)

(41376, 277)
(5, 277)


In [6]:
# Renaming and Droping the columns 
# data_los = data_los[data_los['pred_rev_risk_yr_month'].notnull()]
data_los['order_line_nbr'] = np.where(data_los['order_line_nbr'].isnull(),0,data_los['order_line_nbr'])
data_los['order_line_nbr'] = data_los['order_line_nbr'].astype(int)
data_los['order_line_shipment_nbr'] = np.where(data_los['order_line_shipment_nbr'].isnull(),0,data_los['order_line_shipment_nbr'])

data_los['order_line_shipment_nbr'] = data_los['order_line_shipment_nbr'].astype(int)
data_los.rename(columns={'country_code':'Country', 'order_nbr':'order_nbr', 'order_line_nbr':'order_line_nbr', 'order_line_shipment_nbr':'shipment_nbr', 'mfg_config_id_hypen':'item_hyp', 'pred_rev_risk_code':'RiskCode', 'pred_rev_risk_yr_month':'RiskMonth', 'pred_rev_upside_code':'UpsideCode', 'pred_rev_upside_yr_month':'UpsideMonth' }, inplace=True)
data_los = data_los[['Country','order_nbr','order_line_nbr','shipment_nbr', 'item_hyp', 'RiskCode', 'RiskMonth', 'UpsideCode', 'UpsideMonth']]
#print(data_los)

       Country order_nbr  order_line_nbr  shipment_nbr   item_hyp RiskCode  \
21          IN    801321               2             1                 NaN   
22          IN    803235               2             1                 NaN   
23          IN    803736               2             1                 NaN   
27          IN    804410               2             1                 NaN   
28          IN    801530               2             1                 Red   
...        ...       ...             ...           ...        ...      ...   
34326       XC  18026141               1             1  2062MC277      NaN   
43936       XC  18026140               3             1                 NaN   
44403       XC  18026140               4             1  2062MC277      NaN   
64561       XC  18026140               2             1                 NaN   
186448      XC  18026140               1             1                 NaN   

        RiskMonth UpsideCode  UpsideMonth  
21       202502.0  

In [7]:
data_los.to_csv(r'E:\_Projects\orders\rcv\risk_upside.csv', encoding='utf8', index=False )

In [8]:
# #LOS: apply Hyphen in item
# def apply_hypen(df):
#     #*M* mcids
#     df['mfg_config_id_hypen'] = np.where(df['mfg_config_id'].str.contains('M'), df['mfg_config_id'], '')
#     #4-4-4 kits, cables
#     df['mfg_config_id_hypen'] = np.where( ((df['mfg_config_id'].str.len() == 12) & \
#                                           (df['mfg_config_id_hypen']=='')), \
#     data_los['mfg_config_id'].str[:4] + '-' + data_los['mfg_config_id'].str[4:8] + '-' + data_los['mfg_config_id'].str[8:12], \
#                                          df['mfg_config_id_hypen'] \
#                                         )
#     #*NCR modules
#     df['mfg_config_id_hypen'] = np.where( ((df['mfg_config_id'].str.len() == 13) & \
#                                           (df['mfg_config_id'].str[11:13]=='NCR')), \
#     data_los['mfg_config_id'].str[:3] + '-' + data_los['mfg_config_id'].str[3:10] + '-' + data_los['mfg_config_id'].str[10:14], \
#                                          df['mfg_config_id_hypen'] \
#                                         )
#     #  & (df['mfg_config_id'].str.contains('K') | df['mfg_config_id'].str.contains('C'))

# #     data_los['mfg_config_id_hypen']=''
#     print(df['mfg_config_id_hypen'].unqiue)
# apply_hypen(data_los)

# # print(df['mfg_config_id_hypen'].shape)